In [1]:
# Check GPU availability
!nvidia-smi

Tue Sep 19 05:40:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import files
uploaded = files.upload()

Saving USA-road-d.CAL.gr to USA-road-d.CAL.gr


In [8]:
# Write the modified CUDA code to a file
%%writefile bellman_ford_cuda.cu

// Necessary headers for input-output operations, CUDA runtime, vectors, string manipulations, etc.
#include <cstdio>
#include <iostream>
#include <vector>
#include <fstream>
#include <climits>
#include <cuda_runtime.h>
#include <sstream>
#include <algorithm>

using namespace std;

// CUDA kernel to perform edge relaxation for the Bellman-Ford algorithm.
__global__ void bellmanFordKernel(int* d_V, int* d_E, int* d_W, int* d_dist, int V, int E) {
    // Calculate the global thread index.
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Each thread processes one edge.
    if(idx < E) {
        int u = d_V[idx];
        int v = d_E[idx];
        int w = d_W[idx];

        // Perform relaxation, but ensure we don't overflow.
        if (d_dist[u] != INT_MAX/2 && d_dist[u] + w < d_dist[v]) {
            atomicMin(&d_dist[v], d_dist[u] + w);
        }
    }
}

// Function to write the shortest path distances to an output file.
void writeToFile(const vector<int> &distances, const char *filename) {
    ofstream outfile(filename);
    for (int distance : distances) {
        outfile << distance << endl;
    }
    outfile.close();
}

// Function to read a graph from a .gr file format.
void readGraphFromFile(const char *filename, vector<int> &V, vector<int> &E, vector<int> &W) {
    ifstream infile(filename);
    string line;
    int u, v, w;
    char c;

    while (getline(infile, line)) {
        stringstream ss(line);
        ss >> c;  // Distinguish the line type by the first character.

        // If the line describes an edge, process it.
        if (c == 'a') {
            ss >> u >> v >> w;
            V.push_back(u - 1);  // Convert from 1-based index in file to 0-based index for processing.
            E.push_back(v - 1);
            W.push_back(w);
        }
    }

    infile.close();
}

int main() {
    // Host vectors to store the vertices (V), edges (E), and weights (W) of the graph.
    vector<int> h_V, h_E, h_W;

    // Read the graph data from the file.
    readGraphFromFile("USA-road-d.CAL.gr", h_V, h_E, h_W);

    // Calculate the number of vertices and edges from the data.
    int V = *max_element(h_V.begin(), h_V.end()) + 1;
    int E = h_V.size();

    // Initialize distances with a large but safe value to avoid overflow. Distance to source (vertex 0) is 0.
    vector<int> h_dist(V, INT_MAX/2);
    h_dist[0] = 0;

    // Allocate memory on the GPU for vertices, edges, weights, and distances.
    int *d_V, *d_E, *d_W, *d_dist;
    cudaMalloc(&d_V, E * sizeof(int));
    cudaMalloc(&d_E, E * sizeof(int));
    cudaMalloc(&d_W, E * sizeof(int));
    cudaMalloc(&d_dist, V * sizeof(int));

    // Copy graph data from host to GPU.
    cudaMemcpy(d_V, h_V.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_E, h_E.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_W, h_W.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_dist, h_dist.data(), V * sizeof(int), cudaMemcpyHostToDevice);

    // Set up the CUDA kernel launch parameters.
    int threadsPerBlock = 256;
    int blocksPerGrid = (E + threadsPerBlock - 1) / threadsPerBlock;

    // Launch the Bellman-Ford kernel V-1 times.
    for(int i = 0; i < V - 1; ++i) {
        bellmanFordKernel<<<blocksPerGrid, threadsPerBlock>>>(d_V, d_E, d_W, d_dist, V, E);
        cudaDeviceSynchronize();
    }

    // Copy the computed shortest path distances from GPU back to host.
    cudaMemcpy(h_dist.data(), d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);

    // Write the shortest path distances to an output file.
    writeToFile(h_dist, "shortest_path_distances.txt");

    // Free up the allocated GPU memory.
    cudaFree(d_V);
    cudaFree(d_E);
    cudaFree(d_W);
    cudaFree(d);


    return 0;
}

Overwriting bellman_ford_cuda.cu


In [9]:
!nvcc bellman_ford_cuda.cu -o bellman_ford_cuda
!./bellman_ford_cuda

In [10]:
# Read and print the output file.
!cat shortest_path_distances.txt

Streaming output truncated to the last 5000 lines.
4096098
4096743
4099216
4101672
4100504
4094869
4096251
4100401
4098635
4103490
4103381
4099028
4098813
4096502
4098276
4098358
4099648
4096370
4093598
4090708
4092812
4094995
4093420
4086525
4091829
4092935
4096226
4096991
4093364
4090723
4095552
4096745
4101259
4089365
4094486
4089417
4092198
4093243
4089417
4092878
4095439
4094351
4094719
4098206
4100136
4103312
4101930
4103041
4098219
4105482
4106198
4102018
4104215
4101005
4102689
4102189
4096237
4100146
4098108
4104711
4098077
4099860
4103882
4097206
4098986
4093891
4101584
4101084
4097971
4099650
4099979
4097142
4096325
4098729
4097710
4097124
4096691
4101119
4100353
4100826
4091294
4095371
4095825
4099185
4099905
4098686
4098886
4097667
4097958
4098660
4087681
4089310
4087631
4090332
4087521
4082622
4084063
4084755
4084387
4084639
4084296
4086354
4083099
4083897
4087969
4085576
4080964
4082963
4082850
4081334
4080738
4083540
4080199
4078261
4075502
4081425
4078608
4079215
40813

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Bellman-Ford Algorithm on CUDA: Code Walkthrough

### Headers and Namespaces

```cpp
#include <cstdio>
#include <iostream>
#include <vector>
#include <fstream>
#include <climits>
#include <cuda_runtime.h>
#include <sstream>
#include <algorithm>
```

We start by including necessary header files:
- I/O operations (`<cstdio>`, `<iostream>`, `<fstream>`)
- Common data structures (`<vector>`)
- C++ utilities (`<climits>`, `<sstream>`, `<algorithm>`)
- CUDA-specific operations (`<cuda_runtime.h>`)

### CUDA Kernel

```cpp
__global__ void bellmanFordKernel(int* d_V, int* d_E, int* d_W, int* d_dist, int V, int E) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    if(idx < E) {
        int u = d_V[idx];
        int v = d_E[idx];
        int w = d_W[idx];
        
        if (d_dist[u] != INT_MAX/2 && d_dist[u] + w < d_dist[v]) {
            atomicMin(&d_dist[v], d_dist[u] + w);
        }
    }
}
```

This is the heart of the CUDA implementation: the kernel. Here's what each part does:

1. Calculate the global thread index with `int idx = blockIdx.x * blockDim.x + threadIdx.x;`.
2. Each thread is responsible for relaxing one edge. Thus, check if the current thread's index is within bounds (`idx < E`).
3. Fetch the source vertex (`u`), destination vertex (`v`), and weight (`w`) for the edge the thread is responsible for.
4. If the potential new distance to vertex `v` through vertex `u` is shorter, update it using `atomicMin`.

### Helper Functions

#### Writing to File

```cpp
void writeToFile(const vector<int> &distances, const char *filename) {
    ofstream outfile(filename);
    for (int distance : distances) {
        outfile << distance << endl;
    }
    outfile.close();
}
```

This function writes the calculated shortest-path distances to an output file. It simply iterates through the distances vector and writes each distance to a new line.

#### Reading from File

```cpp
void readGraphFromFile(const char *filename, vector<int> &V, vector<int> &E, vector<int> &W) {
    ifstream infile(filename);
    string line;
    int u, v, w;
    char c;

    while (getline(infile, line)) {
        stringstream ss(line);
        ss >> c;
        
        if (c == 'a') {
            ss >> u >> v >> w;
            V.push_back(u - 1);
            E.push_back(v - 1);
            W.push_back(w);
        }
    }

    infile.close();
}
```

This function reads the graph data from a `.gr` file format. It differentiates edge data lines by checking the starting character (`a`). It then pushes the vertices and weights to their respective vectors.

### Main Function

Inside the `main` function:

1. **Reading the Graph**:

    We declare vectors to hold vertices (`h_V`), edges (`h_E`), and weights (`h_W`), and then we read the graph data from a file.

    ```cpp
    vector<int> h_V, h_E, h_W;
    readGraphFromFile("USA-road-d.CAL.gr", h_V, h_E, h_W);
    ```

2. **Initialize Distances**:

    The number of vertices (`V`) and edges (`E`) are determined, and a vector to hold distances (`h_dist`) is initialized. The source vertex (vertex 0) is initialized with a distance of 0, while others are initialized to half the maximum possible value to avoid overflow.

    ```cpp
    int V = *max_element(h_V.begin(), h_V.end()) + 1;
    int E = h_V.size();
    vector<int> h_dist(V, INT_MAX/2);
    h_dist[0] = 0;
    ```

3. **Memory Allocation & Data Transfer to GPU**:

    Allocate GPU memory for vertices, edges, weights, and distances and then transfer the data from host to GPU.

    ```cpp
    int *d_V, *d_E, *d_W, *d_dist;
    cudaMalloc(&d_V, E * sizeof(int));
    // ... (similar for d_E, d_W, d_dist)
    cudaMemcpy(d_V, h_V.data(), E * sizeof(int), cudaMemcpyHostToDevice);
    // ... (similar for d_E, d_W, d_dist)
    ```
    
4. **Kernel Execution**:

    The Bellman-Ford algorithm is iterative, and the kernel is launched `V-1` times. Each iteration aims to further relax the edges.

    ```cpp
    int threadsPerBlock = 256;
    int blocksPerGrid = (E + threadsPerBlock - 1) / threadsPerBlock;
    for(int i = 0; i < V - 1; ++i) {
        bellmanFordKernel<<<blocksPerGrid, threadsPerBlock>>>(d_V, d_E, d_W, d_dist, V, E);
        cudaDeviceSynchronize();
    }
    ```

5. **Retrieve Results & Cleanup**:

    We then copy the shortest path distances back to the host memory from the GPU and write these distances to an output file. Lastly, we free up the GPU memory.

    ```cpp
    cudaMemcpy(h_dist.data(), d_dist, V * sizeof(int), cudaMemcpyDeviceToHost);
    writeToFile(h_dist, "shortest_path_distances.txt");
    cudaFree(d_V);
    // ... (similar for d_E, d_W, d_dist)
    ```

And that wraps up our CUDA implementation of the Bellman-Ford algorithm!